In [1]:
cd ../../../..

/Users/archnnj/Development/recsys/recsys_polimi_challenge_2018/repo


In [2]:

import sys
# sys.path.append('src/libs/RecSys_Course_2018')
# sys.path.append('/home/stefano/git/recsys/recsys_challenge/src/libs/RecSys_Course_2018')
# sys.path.append('/home/stefano/git/recsys/recsys_challenge')

sys.path.append("src/libs/RecSys_Course_2018/SequenceAware/sars_tutorial_master") # go to parent dir
sys.path.append("src/libs/RecSys_Course_2018") # go to parent dir

import numpy as np
import pandas as pd
# import matplotlib.pyplot as pyplot
import src.utils.build_icm as build_icm
import scipy.sparse as sps
from src.utils.data_splitter import train_test_holdout, train_test_user_holdout, train_test_row_holdout
# from src.utils.evaluation import evaluate_algorithm, evaluate_algorithm_recommendations
# from src.utils.top_n_idx_sparse import top_n_idx_sparse
# from src.utils.Compute_Similarity_Python import Compute_Similarity_Python
# from src.libs.similarity import cosine
# import src.utils.similarity_wrapper as sim
# import time

import traceback, os
import datetime

# import sys
# sys.path.append('src/libs/RecSys_Course_2018')


from src.recommenders.HybridLinCombItemSimilarities import HybridLinCombItemSimilarities
from src.recommenders.HybridLinCombEstRatings import HybridLinCombEstRatings

from Base.NonPersonalizedRecommender import TopPop, Random
from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from SLIM_BPR.Cython.UserSLIM_BPR_Cython import UserSLIM_BPR_Cython
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from GraphBased.P3alphaRecommender import P3alphaRecommender
from GraphBased.RP3betaRecommender import RP3betaRecommender
from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from MatrixFactorization.PureSVD import PureSVDRecommender
from FW_Similarity.CFW_D_Similarity_Linalg import CFW_D_Similarity_Linalg
from LightFM.LightFMRecommender import LightFMRecommender

from src.recommenders.ImplicitALSRecommender import ImplicitALSRecommender
from src.recommenders.ImplicitBPRRecommender import ImplicitBPRRecommender

from SequenceAware.sars_tutorial_master.recommenders.MixedMarkovRecommender import MixedMarkovChainRecommender

/Users/archnnj/anaconda3/envs/recsys_python36/lib/python3.6/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [6]:
print("Starting importing everything")
from src.recommenders.ItemCFKNNRecommender import ItemCFKNNRecommender
from src.recommenders.ItemCBFKNNRecommender import ItemCBFKNNRecommender
from src.recommenders.P3AlphaRecommender import P3AlphaRecommender
from src.recommenders.UserCFKNNRecommender import UserCFKNNRecommender
from src.recommenders.UserCBFKNNRecommender import UserCBFKNNRecommender
from src.recommenders.ImplicitALSRecommender import ImplicitALSRecommender
import src.utils.build_icm as build_icm
import time

JUPYTER = False
#sys.path.append('src/data/')

# #### Load data
if JUPYTER:
    # Jupyter
    tracks_csv_file = "../../../data/tracks.csv"
    interactions_csv_file = "../../../data/train.csv"
    playlist_id_csv_file = "../../../data/target_playlists.csv"
    sequential_csv_file = "../../../data/train_sequential.csv"
else:
    # PyCharm
    tracks_csv_file = "data/tracks.csv"
    interactions_csv_file = "data/train.csv"
    playlist_id_csv_file = "data/target_playlists.csv"
    sequential_csv_file = "data/train_sequential.csv"

tracks_df = pd.read_csv(tracks_csv_file)
interactions_df = pd.read_csv(interactions_csv_file)
playlist_id_df = pd.read_csv(playlist_id_csv_file)
train_sequential_df = pd.read_csv(sequential_csv_file)
userList = interactions_df["playlist_id"]
itemList = interactions_df["track_id"]
ratingList = np.ones(interactions_df.shape[0])
targetsList = playlist_id_df["playlist_id"]
targetsListOrdered = targetsList[:5000].tolist()
targetsListCasual = targetsList[5000:].tolist()
userList_unique = pd.unique(userList)
itemList_unique = tracks_df["track_id"]
numUsers = len(userList_unique)
numItems = len(itemList_unique)
numberInteractions = interactions_df.size

# Build ICM
#ICM_all = build_icm.build_icm(tracks_df, split_duration_lenght=800, feature_weights={'albums': 1, 'artists': 0.5, 'durations': 0.1})
ICM_all = build_icm.build_icm(tracks_df)

IDF_ENABLED = True

if IDF_ENABLED:
    num_tot_items = ICM_all.shape[0]
    # let's count how many items have a certain feature
    items_per_feature = (ICM_all > 0).sum(axis=0)
    IDF = np.array(np.log(num_tot_items / items_per_feature))[0]
    ICM_idf = ICM_all.copy()
    # compute the number of non-zeros in each col
    # NOTE: this works only if X is instance of sparse.csc_matrix
    col_nnz = np.diff(sps.csc_matrix(ICM_idf).indptr)
    # then normalize the values in each col
    ICM_idf.data *= np.repeat(IDF, col_nnz)
    ICM_all = ICM_idf  # use IDF features

# #### Build URM

URM_all = sps.coo_matrix((ratingList, (userList, itemList)))
URM_all_csr = URM_all.tocsr()

URM_IDF_ENABLED = False

if URM_IDF_ENABLED:
    num_tot_items = URM_all.shape[0]
    # let's count how many items have a certain feature
    items_per_feature = (URM_all > 0).sum(axis=0)
    IDF = np.array(np.log(num_tot_items / items_per_feature))[0]
    URM_idf = URM_all.copy()
    # compute the number of non-zeros in each col
    # NOTE: this works only if X is instance of sparse.csc_matrix
    col_nnz = np.diff(sps.csc_matrix(URM_idf).indptr)
    # then normalize the values in each col
    URM_idf.data *= np.repeat(IDF, col_nnz)
    URM_all = URM_idf  # use IDF features

# #### Train/test split: ratings and user holdout

seed = 0
usersNonOrdered = [i for i in userList_unique if i not in targetsListOrdered]
URM_train, URM_valid_test_pred = train_test_row_holdout(URM_all, targetsListOrdered, train_sequential_df,
                                                        train_perc=0.8,
                                                        seed=seed, targetsListOrdered=targetsListOrdered,
                                                        nnz_threshold=2)
URM_test_known = None

URM_train = URM_train
URM_validation = URM_valid_test_pred

print("Starting initing the single recsys")

# N_cbf = 6
# N_cf = 40
# N_p3a = 3
# N_ucf = 20
# N_ucbf = 8
# N_rp3b = 3
N_cbf = 2
N_cf = 6
N_p3a = 0
N_ucf = 2
N_ucbf = 1
N_rp3b = 1
N_slim = 1
N_mix_markov = 1
N_als = 1
N_hyb_item_sim = 0
N_pure_svd = 0
N_hyb = N_cbf + N_cf + N_p3a + N_ucf + N_ucbf + N_rp3b + N_slim + N_mix_markov + N_als + N_hyb_item_sim + N_pure_svd
recsys = []
for i in range(N_cbf):
    recsys.append(ItemCBFKNNRecommender(URM_train, ICM_all))
for i in range(N_cf):
    recsys.append(ItemCFKNNRecommender(URM_train))
for i in range(N_p3a):
    recsys.append(P3AlphaRecommender(URM_train))
for i in range(N_ucf):
    recsys.append(UserCFKNNRecommender(URM_train))
for i in range(N_ucbf):
    recsys.append(UserCBFKNNRecommender(URM_train, ICM_all))
for i in range(N_rp3b):
    recsys.append(RP3betaRecommender(URM_train))
for i in range(N_slim):
    recsys.append(SLIM_BPR_Cython(URM_train))
for i in range(N_mix_markov):
    recsys.append(MixedMarkovChainRecommender(URM_train, train_sequential_df, targetsListOrdered, min_order=1, max_order=1))
for i in range(N_als):
    recsys.append(ImplicitALSRecommender(URM_train))
for i in range(N_pure_svd):
    recsys.append(PureSVDRecommender(URM_train))

recsys_params = list(zip(np.linspace(10, 70, N_cbf).tolist(), [4] * N_cbf))
recsys_params2 = list((zip(np.linspace(5, 400, N_cf).tolist(), [12] * N_cf)))
recsys_params3 = list((zip(np.linspace(99, 101, N_p3a).tolist(), [1] * N_p3a)))
recsys_params4 = list((zip(np.linspace(10, 180, N_ucf).tolist(), [2] * N_ucf)))
recsys_params5 = list((zip(np.linspace(170, 180, N_ucbf).tolist(), [5] * N_ucbf)))
recsys_params6 = list((zip(np.linspace(99, 101, N_rp3b).tolist(), [0] * N_rp3b)))

print("Starting fitting single recsys")
t = time.time()
for i in range(N_cbf):
    # print("Training system {:d}...".format(i))
    topK = recsys_params[i][0]
    shrink = recsys_params[i][1]
    recsys[i].fit(topK=topK, shrink=shrink, type="tanimoto")
for i in range(N_cf):
    # print("Training system {:d}...".format(i+N_cbf))
    topK = recsys_params2[i][0]
    shrink = recsys_params2[i][1]
    recsys[i + N_cbf].fit(topK=topK, shrink=shrink, type="cosine", alpha=0.3)
for i in range(N_p3a):
    # print("Training system {:d}...".format(i+N_cbf))
    topK = recsys_params3[i][0]
    shrink = recsys_params3[i][1]
    recsys[i + N_cbf + N_cf].fit(topK=topK, shrink=shrink, alpha=0.31)
for i in range(N_ucf):
    # print("Training system {:d}...".format(i+N_cbf))
    topK = recsys_params4[i][0]
    shrink = recsys_params4[i][1]
    recsys[i + N_cbf + N_cf + N_p3a].fit(topK=topK, shrink=shrink, type="jaccard")
for i in range(N_ucbf):
    # print("Training system {:d}...".format(i+N_cbf))b
    topK = recsys_params5[i][0]
    shrink = recsys_params5[i][1]
    recsys[i + N_cbf + N_cf + N_p3a + N_ucf].fit(topK=topK, shrink=shrink, type="tanimoto")
for i in range(N_rp3b):
    # print("Training system {:d}...".format(i+N_cbf))b
    topK = int(recsys_params6[i][0])
    shrink = recsys_params6[i][1]
    recsys[i + N_cbf + N_cf + N_p3a + N_ucf + N_ucbf].fit(topK=topK, alpha=0.5927789387679869,
                                                          beta=0.009260542392306892)

Starting importing everything
Starting initing the single recsys
SLIM_BPR_Cython: Estimated memory required for similarity matrix of 20635 items is 1703.21 MB
Starting fitting single recsys
SLIM_BPR_Recommender: Loading model from file 'result_experiments/hyb_est_ratings_6/SLIM_BPR_rw_300'


FileNotFoundError: [Errno 2] No such file or directory: 'result_experiments/hyb_est_ratings_6/SLIM_BPR_rw_300'

In [11]:

# slims_dir_old = "result_experiments/hyb_est_ratings_4/"
# slims_dir = "result_experiments/hyb_est_ratings_6/"
# recsys[-3].loadModel(slims_dir_old, "SLIM_BPR_Recommender_best_model_300")
# recsys[-2].loadModel(slims_dir, "SLIM_BPR_rw_300")
# print("Load complete of slim bpr")

# load slim bpr
slims_dir = " src/libs/RecSys_Course_2018/result_experiments/hyb_est_ratings_6/"
recsys[-3].loadModel(slims_dir, "SLIM_BPR_rw_300")
print("Load complete of slim bpr")
el_t = time.time() - t
print("Done. Elapsed time: {:02d}:{:06.3f}".format(int(el_t / 60), el_t - 60 * int(el_t / 60)))

# load mix markov
mix_markov_dir = "dump/"
recsys[-2].loadModel(mix_markov_dir, "dump_MixedMarkovChainRecommender_ord_1_1")
print("Load complete of mix markov")
el_t = time.time() - t
print("Done. Elapsed time: {:02d}:{:06.3f}".format(int(el_t / 60), el_t - 60 * int(el_t / 60)))

# print("Starting fitting als")
# recsys[-1].fit(alpha=15, factors=495, regularization=0.04388, iterations=20)
# print("Ended fitting als")

# print("Starting fitting PureSVD")
# recsys[-1].fit(num_factors=165)
# print("PureSVD fitted")

# print("Starting recommending svd")
# svd_est = recsys[-1].compute_score_SVD(userList_unique, 160)
# print("Ended recommending svd")

print("Starting recommending the est_ratings")
t2 = time.time()
recsys_est_ratings = []
for i in range(0, N_hyb - 1):
    if i < N_cbf + N_cf + N_p3a + N_ucf + N_ucbf:
        recsys_est_ratings.append(recsys[i].estimate_ratings(userList_unique, 160))
    elif i < N_cbf + N_cf + N_p3a + N_ucf + N_ucbf + N_rp3b + N_slim: # + N_mix_markov + N_als + N_hyb_item_sim + N_pure_svd:
        recsys_est_ratings.append(recsys[i].compute_item_score(userList_unique, 160))
    elif i < N_cbf + N_cf + N_p3a + N_ucf + N_ucbf + N_rp3b + N_slim + N_mix_markov:
        recsys_est_ratings.append(recsys[i].compute_item_score(targetsListOrdered, 160))
    else:
        recsys_est_ratings.append(recsys[i].compute_item_score(userList_unique, 160))
el_t = time.time() - t2
print("Done. Elapsed time: {:02d}:{:06.3f}".format(int(el_t / 60), el_t - 60 * int(el_t / 60)))

# print("Recommending als")
# t2 = time.time()
# # recsys_est_ratings.append(recsys[-1].estimate_ratings(userList_unique, 160))
# recsys_est_ratings.append(recsys[-1].loadEstRatings(slims_dir, "ALS_rw_est_rat")[0])
# el_t = time.time() - t2
# print("ALS done. Elapsed time: {:02d}:{:06.3f}".format(int(el_t / 60), el_t - 60 * int(el_t / 60)))

print("Recommending als")
t2 = time.time()
# recsys_est_ratings.append(recsys[-1].estimate_ratings(userList_unique, 160))
slims_dir = " src/libs/RecSys_Course_2018/result_experiments/hyb_est_ratings_6/"
recsys_est_ratings.append(recsys[-1].loadEstRatings(slims_dir, "ALS_rw_est_rat")[0])
el_t = time.time() - t2
print("ALS done. Elapsed time: {:02d}:{:06.3f}".format(int(el_t / 60), el_t - 60 * int(el_t / 60)))

SLIM_BPR_Recommender: Loading model from file ' src/libs/RecSys_Course_2018/result_experiments/hyb_est_ratings_6/SLIM_BPR_rw_300'


FileNotFoundError: [Errno 2] No such file or directory: ' src/libs/RecSys_Course_2018/result_experiments/hyb_est_ratings_6/SLIM_BPR_rw_300'

In [14]:
ls src/libs/RecSys_Course_2018/result_experiments/hyb_est_ratings_6

ALS_complete_est_rat*  SLIM_BPR_complete*     SLIM_BPR_valid_06_300*
ALS_rw_est_rat*        SLIM_BPR_rw_300*
